In [ ]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sea
import cv2 as op
import tensorflow as tf
from keras.applications.mobilenet import preprocess_input
from tensorflow.keras.utils import to_categorical

np.__version__

'1.22.4'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/MyDrive/images'

label_id = 0
filepath, outcome = [], []
labels = {}

for label in tqdm(os.listdir(PATH)):
  labels[label] = label_id
  for path in os.listdir(PATH + "/" + label):
    filepath.append(PATH + "/" + label + "/" + path)
    outcome.append(label_id)
  label_id += 1


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame({
    "filepath" : filepath,
    "outcome" : outcome
})

df

,filepath,outcome
0,/content/drive/MyDrive/images/metal/metal0.png,0
1,/content/drive/MyDrive/images/metal/metal1.png,0
2,/content/drive/MyDrive/images/metal/metal2.png,0
3,/content/drive/MyDrive/images/metal/metal3.png,0
4,/content/drive/MyDrive/images/metal/metal4.png,0
...,...,...
1013,/content/drive/MyDrive/images/jazz/jazz95.png,9
1014,/content/drive/MyDrive/images/jazz/jazz97.png,9
1015,/content/drive/MyDrive/images/jazz/jazz96.png,9
1016,/content/drive/MyDrive/images/jazz/jazz98.png,9


In [ ]:
X, y = [], []

def load_image(path):
  img = op.imread(path)
  img = op.resize(img, (224, 224))
  img = op.cvtColor(img, op.COLOR_BGR2RGB)
  img = img/255
  return img

# img = load_image(df.iloc[0,0])
for i in tqdm(range(len(df))):
  X.append(load_image(df.iloc[i, 0]))
  y.append(df.iloc[i, 1])

  0%|          | 0/1018 [00:00<?, ?it/s]

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((814, 224, 224, 3), (204, 224, 224, 3), (814,), (204,))

In [ ]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:

import tensorflow
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input
from keras.models import Model

def make_model():
  inp = Input(shape = (224, 224, 3))
  x = Conv2D(16, (3,3), activation = 'relu')(inp)
  x = MaxPool2D(2)(x)
  x = Dropout(0.2)(x)

  x = Conv2D(32, (3,3), activation = 'relu')(x)
  x = MaxPool2D(2)(x)
  x = Dropout(0.2)(x)

  x = Conv2D(64, (3,3), activation = 'relu')(x)
  x = MaxPool2D(2)(x)
  x = Dropout(0.2)(x)

  x = Conv2D(32, (3,3), activation = 'relu')(x)
  x = MaxPool2D(2)(x)
  x = Dropout(0.2)(x)

  x = Conv2D(64, (3,3), activation = 'relu')(x)
  x = MaxPool2D(2)(x)
  x = Dropout(0.2)(x)

  x = Flatten()(x)
  x = Dense(512, activation = 'relu')(x)
  x = Dropout(0.2)(x)
  out = Dense(10, activation = 'softmax')(x)
  
  model = Model(inputs = inp, outputs = out)

  return model



In [ ]:
model = make_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 111, 111, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                         

In [ ]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [ ]:
model.fit(X_train, y_train, epochs = 30, batch_size = 32, validation_data = (X_test, y_test))

Epoch 1/30
26/26 [==============================] - 15s 100ms/step - loss: 2.2091 - accuracy: 0.1634 - val_loss: 2.0971 - val_accuracy: 0.2108
Epoch 2/30
26/26 [==============================] - 1s 52ms/step - loss: 1.9753 - accuracy: 0.2076 - val_loss: 2.0318 - val_accuracy: 0.2549
Epoch 3/30
26/26 [==============================] - 1s 47ms/step - loss: 1.8650 - accuracy: 0.2789 - val_loss: 1.9858 - val_accuracy: 0.2108
Epoch 4/30
26/26 [==============================] - 1s 49ms/step - loss: 1.8517 - accuracy: 0.2936 - val_loss: 1.9412 - val_accuracy: 0.2451
Epoch 5/30
26/26 [==============================] - 1s 49ms/step - loss: 1.8473 - accuracy: 0.2899 - val_loss: 1.9481 - val_accuracy: 0.2451
Epoch 6/30
26/26 [==============================] - 1s 49ms/step - loss: 1.7700 - accuracy: 0.3231 - val_loss: 1.9483 - val_accuracy: 0.2794
Epoch 7/30
26/26 [==============================] - 1s 48ms/step - loss: 1.7122 - accuracy: 0.3489 - val_loss: 1.8153 - val_accuracy: 0.2696
Epoch 8/30
